In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy
from decimal import *
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
import threading
import numpy as np
import datetime
import itertools
import plotly.express as px
import plotly.graph_objects as go
from pyvis.network import Network

In [2]:
numNodes = 10

In [40]:
def setupStake(
        # Number of nodes in the network. This number can be changed to test with small and large network size
        numNodes=numNodes,
        # the minimum number of channels a node can open
        minChannelsPerNode=2,
        # the maximum number of channels a node can open 
        maxChannelsPerNode=10,
        minFundsPerNode=10,
        maxFundsPerNode=100,
        tokensPerTicket=0.1
    ):

    stake = [[0 for i in range(numNodes)] for j in range(numNodes)]
    for x in range(numNodes-1):
        # each node is given a random funding amount
        myFunds = numpy.random.rand() * (maxFundsPerNode - minFundsPerNode) + minFundsPerNode

        # get random number of channels per node
        myChannels = int(numpy.random.rand() * (maxChannelsPerNode - minChannelsPerNode + 1) + minChannelsPerNode)

        # This value represents the amount each node stakes in their channel
        # It is computed as the number of funds a node has divided by number of channels they open
        stakePerChannel = myFunds / myChannels
        stakePerChannel = int(stakePerChannel / tokensPerTicket) * tokensPerTicket

        # fund channels by writing into stake matrix
        for c in range(myChannels):
            # TODO: this does not prevent a node from opening a channel to the same counterparty multiple times
            counterparty = int(numpy.random.rand() * (numNodes - 1))

            # cannot open channel to self - keep diagonal of matrix at 0
            if counterparty >= x:
                counterparty = counterparty + 1
            stake[x][counterparty] = stakePerChannel
            stake = [[Decimal(i) for i in j] for j in stake]

    return stake

In [41]:
initial_stake = setupStake()

In [42]:
def calcImportance(stake):

    # get total stake per node
    stakePerNode = [Decimal(sum(e)) for e in stake]

    totalStake = sum(stakePerNode)

    n = len(stake)
    weight = [0] * n
    importance = [0] * n
    for x in range(n):
        for y in range(n):
            # weight of a node is 0 if its stake is equal to 0
            #weight(channel) = sqrt(balance(channel) / stake(channel.source) * stake(channel.destination))
            weight[y] += 0 if stakePerNode[y]==0 else numpy.sqrt(stake[y][x]/stakePerNode[y] * stakePerNode[x])

    #print("Weighted downstream stake per node p(n) = ", weight)
    importanceList = numpy.array(weight) * numpy.array(stakePerNode)
    return importanceList;

In [43]:
initial_importance = calcImportance(initial_stake)

In [44]:
def selectChannel(weights, weightIndexToNodeLUT):
    """
    Randomly selects a counterparty based on weights according to a LUT
    Returns the node id (or -1 if none could be found)
    Parameters
    ----------
    weights: a list of weights to be used for normalizing the random selection
    weightIndexToNodeLUT: a LUT translating the weights list to node ids
    """
    rand = numpy.random.rand()
    totalWeight = sum(weights)
    sumWeights = 0
    counterparty = -1
    for i in enumerate(weights):
        sumWeights += i[1]
        if totalWeight == 0:
           sumWeights = 0
        else:
            if sumWeights / totalWeight >= rand:
               counterparty = weightIndexToNodeLUT[i[0]]
               break;
    return counterparty

In [45]:
def randomPickWeightedByImportance(importance):
    channel = selectChannel(importance, [i for i in range(len(importance))])
    return channel

In [46]:
def openChannels(stake, importance):
        #print("opening channels")
        numOpenChannels = 0
        channelcount = 1
        ctn = numNodes-1
        openchannelid = [ctn]
        tokensPerChannel = 10
        # add own id to avoid self-staking
        while (numOpenChannels < channelcount):

            # do not open channels to same counterparty multiple times
            tmpImportance = list(importance)
            for i in range(len(importance)):
                if i in openchannelid:
                    tmpImportance[i] = 0

            newChannel = randomPickWeightedByImportance(tmpImportance)
            
            if (newChannel == -1):
                print("ERROR: could not find a counterparty to open the channel to!")
                # TODO: here we should try again but this time including 0 importance nodes
            else:
                #print("opening channel to node ", newChannel)
                stake[ctn][newChannel] = stake[ctn][newChannel] + 10
                openchannelid.append(newChannel)
            # if opening channels didn't work we still want this loop to terminate
            numOpenChannels = numOpenChannels + 1
        
        return openchannelid

In [47]:
def sendPacket(stake, importance):
        #print("sending packet")

        # persist importance list between attempts so that dead ends can be removed
        importanceAttempts = importance.copy()
        attemptsPerTick = 10
        hops = 3
        numPlayers = 4
        ctNodeId = numNodes-1

        for a in range(attemptsPerTick):
            nextNodeIndex = ctNodeId
            pathIndices = [nextNodeIndex]

            # try to find a path
            for j in range(hops):
                # reset importance
                importanceTmp = importanceAttempts.copy()
                #hoprsim.printArray1d(importanceTmp, 1)

                # remove importance entries for nodes to which current hop has no open channels
                # this is used in the path selection for the next hop
                for i in range(numPlayers):
                    if stake[nextNodeIndex][i] == 0 :
                        importanceTmp[i] = 0

                # prevent loops in path by removing existing nodes on path from list
                for i in pathIndices:
                    importanceTmp[i] = 0

                #hoprsim.printArray1d(importanceTmp, 1)
                nextNodeIndex = randomPickWeightedByImportance(importanceTmp)
                if nextNodeIndex == -1:
                    break # stop looking for path if no next node could be found
                pathIndices.append(nextNodeIndex)

            #print("Found path: ", pathIndices)
            if len(pathIndices) < hops + 1:
                print('ignored path')
            else:
                return pathIndices
            # then facilitate they payout to each node
            # but only as long as the edge is valid (new earnings + existing earnings <= counter party stake)

In [48]:
#The function iterates through the opening and packet sending functions using the same initial stake. 
#The output of the function is a dataframe, that sums up the iterations by node (the index refers to the node number)

def iteration_with_fixed_stake(stake, importance):    
    opening_count = []
    opening_list = []
    sending_count = []
    sending_list = []
    
    for _ in range(1000):
        #running the openChannel function 1000 times 
        #creating a list from the outputs
        opening_channel_to = openChannels(stake, importance)
        opening_count.append(opening_channel_to)
        
        #running the sendPacket function 1000 times 
        #creating a list from the outputs
        sending_packet_to = sendPacket(stake, importance)
        sending_count.append(sending_packet_to)
    
    #orginizing the list in a dataframe - where each opening/sending position refers to a column 
    opening_pd = pd.DataFrame(opening_count)
    sending_pd = pd.DataFrame(sending_count)

    #summing up the opening appearance by node
    for h in range(len(opening_pd.columns)):
        #counting the value by opening positions (e.g. - how many times showep up the given node as the first node of openings)
        openings = opening_pd[h].value_counts()
        opening_list.append(openings)
    
    #organizing the list in a new dataframe
    opening_list = pd.concat(opening_list).to_frame().reset_index()
    #renaming the columns 
    opening_list.columns = ['node_opening_to', 'sum_opening']
      
    #summing up the sending appearance by node
    for k in range(len(sending_pd.columns)):
        #counting the value by opening positions (e.g. - how many times showep up the given node as the first node of openings)
        sendings = sending_pd[k].value_counts()
        sending_list.append(sendings)
    
    #organizing the list in a new dataframe
    sending_list = pd.concat(sending_list).to_frame().reset_index()
    #renaming the columns
    sending_list.columns = ['node_sending_to', 'sum_sending']
    
    
    #summing up by nodes
    opening_list = opening_list.groupby(['node_opening_to']).sum()
    sending_list = sending_list.groupby(['node_sending_to']).sum()
    stake_ct = stake[numNodes-1]
    
    #creating a dataframe from the output
    output = pd.concat([opening_list, sending_list], axis=1)
    output['reward'] = stake_ct
    
    return output

In [49]:
initial_packet_result = iteration_with_fixed_stake(initial_stake, initial_importance)
initial_packet_result

,sum_opening,sum_sending,reward
0,64,184,640
1,32,84,320
2,231,298,2310
3,112,331,1120
4,30,194,300
5,180,583,1800
6,174,605,1740
7,149,568,1490
8,28,153,280
9,1000,1000,0


In [50]:
def calculating_gini_index(initial_packet_result):
    
    df_0 = pd.DataFrame([[0,0,0]], columns=['sum_opening', 'sum_sending', 'reward'])
    initial_packet_result = pd.concat([df_0, initial_packet_result])
    initial_packet_result = initial_packet_result.iloc[:-1 , :]
    
    #orginizing the packet receiving in growing order
    gini_df=initial_packet_result.sort_values(by=["sum_sending"]).reset_index()
    
    
    #creating a column for the cumulative sums
    gini_df['cumul_sum'] = gini_df["sum_sending"].cumsum()
    gini_df = gini_df.reset_index()
    
    #renamind the indexes
    gini_df = gini_df.rename(columns={"level_0": "packets_ingrowing_order", "index": "number_of_node"})
    
    #absolute sum of packets
    absolute_sum = gini_df['cumul_sum'][-1:]
    
    #calculating the cumulative sum's proportions
    gini_df['cumul_sum_proportion_sum'] = (gini_df["sum_sending"]/float(absolute_sum)).cumsum()
    
    #calculating the "lorenz area's A"
    area_a = [gini_df['cumul_sum_proportion_sum'][0]/2*1/(numNodes-1)]
    
    for z in range(len(gini_df)-1):
        b = (gini_df['cumul_sum_proportion_sum'][z+1]+gini_df['cumul_sum_proportion_sum'][z])/2*1/(numNodes-1)
        area_a.append(b)
    
    #adding it to the dataframe
    gini_df['area_a'] = area_a
    
    #gini coefficient based on the lorenz's area
    gini_c = (0.5-sum(gini_df['area_a']))/0.5
    print("Gini coefficient: " + str(gini_c))
    
    #plot - Lorenz line
    fig = px.line(gini_df, x="packets_ingrowing_order", y="cumul_sum_proportion_sum", text="number_of_node", width=800, height=800)
    fig.update_traces(textposition="bottom right")
    fig.show()
    
    return gini_df 

In [51]:
gini_initial = calculating_gini_index(initial_packet_result)
gini_initial

Gini coefficient: 0.316962962962963


,packets_ingrowing_order,number_of_node,sum_opening,sum_sending,reward,cumul_sum,cumul_sum_proportion_sum,area_a
0,0,0,0,0,0,0,0.000000,0.000000
1,1,1,32,84,320,84,0.028000,0.001556
2,2,8,28,153,280,237,0.079000,0.005944
3,3,0,64,184,640,421,0.140333,0.012185
4,4,4,30,194,300,615,0.205000,0.019185
5,5,2,231,298,2310,913,0.304333,0.028296
6,6,3,112,331,1120,1244,0.414667,0.039944
7,7,7,149,568,1490,1812,0.604000,0.056593
8,8,5,180,583,1800,2395,0.798333,0.077907
9,9,6,174,605,1740,3000,1.000000,0.099907


In [52]:
def recalculating_stakes_with_importance(stake, importance):
    #calculating sum of the original stake
    sum_stake = sum([sum(stake[li]) for li in range(numNodes)])
    
    #creating an empy stake matrix with the original shape
    stake_mod = [[0 for i in range(numNodes)] for j in range(numNodes)]
    stake_mod_i = [[0 for i in range(numNodes)] for j in range(numNodes)]
    
    #populating it with recalculation
    for p in range(numNodes):
        for r in range(numNodes):
            if p == r:
                stake_mod[p][r] = 0
            else:
                stake_mod[p][r] = importance[r]/(sum(importance)-importance[p])*sum(stake[p])
                
    return stake_mod

In [84]:
#ensuring that the stake matrix is the original one - (CT node's stakes = 0)
for j in range(len(initial_stake)): initial_stake[numNodes-1][j] = 0

initial_stake = [[Decimal(i) for i in j] for j in initial_stake]

In [85]:
initial_stake

[[Decimal('0'),
  Decimal('0'),
  Decimal('0'),
  Decimal('0'),
  Decimal('0'),
  Decimal('8.5999999999999996447286321199499070644378662109375'),
  Decimal('0'),
  Decimal('0'),
  Decimal('8.5999999999999996447286321199499070644378662109375'),
  Decimal('0')],
 [Decimal('0'),
  Decimal('0'),
  Decimal('0'),
  Decimal('2.9000000000000003552713678800500929355621337890625'),
  Decimal('2.9000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('0'),
  Decimal('2.9000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('0')],
 [Decimal('7.4000000000000003552713678800500929355621337890625'),
  Decimal('7.4000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('7.4000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('7.4000000000000003552713678800500929355621337890625'),
  Decimal('0'),
  Decimal('0'),
  Decimal('7.4000000000000003552713678800500929355621337890625'),
  Decimal('7.400000000000000355271

In [62]:
recalculated_stake = recalculating_stakes_with_importance(initial_stake_pure, initial_importance)

In [63]:
recalculated_importance = calcImportance(recalculated_stake)

In [64]:
recalculated_packet_result = iteration_with_fixed_stake(recalculated_stake, recalculated_importance)

In [65]:
gini_recalculated = calculating_gini_index(recalculated_packet_result)
gini_recalculated

Gini coefficient: 0.16266666666666663


,packets_ingrowing_order,number_of_node,sum_opening,sum_sending,reward,cumul_sum,cumul_sum_proportion_sum,area_a
0,0,0,0,0,0,0,0.000000,0.000000
1,1,3,67,190,670.0000000000000000000000000,190,0.063333,0.003519
2,2,7,70,251,700.0000000000000000000000000,441,0.147000,0.011685
3,3,2,83,276,830.0000000000000000000000000,717,0.239000,0.021444
4,4,1,90,297,900.0000000000000000000000000,1014,0.338000,0.032056
5,5,0,93,304,930.0000000000000000000000000,1318,0.439333,0.043185
6,6,6,117,352,1170.000000000000000000000000,1670,0.556667,0.055333
7,7,4,123,359,1230.000000000000000000000000,2029,0.676333,0.068500
8,8,5,137,396,1370.000000000000000000000000,2425,0.808333,0.082481
9,9,8,220,575,2200.000000000000000000000000,3000,1.000000,0.100463


In [66]:
#drawing comparative plot from the result of the inital and recalculated stakes
fig = go.Figure()
fig.add_trace(go.Scatter(x=gini_initial['packets_ingrowing_order'], 
                         y=gini_initial['cumul_sum_proportion_sum'], 
                        mode='lines+text',
                        name='Initial stake', 
                        text=gini_initial['number_of_node'], 
                        textposition="bottom right"))
fig.add_trace(go.Scatter(x=gini_recalculated['packets_ingrowing_order'], 
                         y=gini_recalculated['cumul_sum_proportion_sum'],
                        mode='lines+text',
                        name='Recalculated stake',
                        text=gini_recalculated['number_of_node'], 
                        textposition="top left"))
fig.add_trace(go.Scatter(x=[0,(numNodes-1)/2,numNodes-1], 
                         y=[0,0.5,1],
                        mode='lines+text',
                        name='Equality line ',
                        ))

fig.update_layout(autosize=False, width=1000, height=1000,)

fig.show()

In [67]:
# the function creates a dataframe, which can be easily used to generate a network graph 
my_network = Network(heading='', notebook=True, height='700px', width='100%', directed=True, bgcolor="#222222", font_color='yellow')

def network_visualisation_dataframe(stake):
    target_list = []
    source_list = []
    
    # from array it creates a single column list
    def flatten(t):
        return [item for sublist in t for item in sublist]
    
    #empty list for the target variable
    for j in range(numNodes):
        l = [j for j in range(numNodes)]
        target_list.append(l)
    
    #empty list for the source variable
    for g in range(numNodes):
        h = [g]*numNodes
        source_list.append(h)
     
    #creates dataframe from the stake matrix, and from the source, target list 
    weight_raw = pd.DataFrame(flatten(stake))
    weight = weight_raw[0].apply(lambda x: float(x))
    target = pd.DataFrame(flatten(target_list))
    source = pd.DataFrame(flatten(source_list))
    
    #merging, renaming the dataframes
    df_merge = pd.concat([source, target, weight], axis=1)
    df_merge.columns=['source', 'target', 'weight']
    
    #removing the channels without weight
    df_to_draw = (df_merge[df_merge['weight'] > 0]).reset_index(drop=True)
    
    return df_to_draw

In [74]:
df_initial = network_visualisation_dataframe(initial_stake)

In [75]:
df_recalculated = network_visualisation_dataframe(recalculated_stake)

In [81]:
recalculated_stake

[[0,
  Decimal('1.613948619407206153815658286'),
  Decimal('9.731920512407976783872240089'),
  Decimal('4.629529132719877033302592344'),
  Decimal('1.637179628151134407309030818'),
  Decimal('6.941012510274075351527754142'),
  Decimal('7.864650691777860819380584559'),
  Decimal('6.275966601827912477145832475'),
  Decimal('1.405792303433958394731778799'),
  Decimal('0.00')],
 [Decimal('2.401879998621732342652295943'),
  0,
  Decimal('8.597062820312244850493671804'),
  Decimal('4.089670967997930575499995197'),
  Decimal('1.446265009399277480962728701'),
  Decimal('6.131607888835360738465330093'),
  Decimal('6.947538871779883796088958564'),
  Decimal('5.544114243977320593850882761'),
  Decimal('1.241860199076248200900665420'),
  Decimal('0.00')],
 [Decimal('2.958316262709005343174954338'),
  Decimal('1.756040577820512035899402262'),
  0,
  Decimal('5.037112653712778169190743362'),
  Decimal('1.781316843450966091191345507'),
  Decimal('7.552098916059625106071877758'),
  Decimal('8.55705415

In [76]:
#my_network is the basic layout of the graph
my_network = Network(heading='', notebook=True, height='700px', width='100%', directed=True, bgcolor="#222222", font_color='yellow')

def drawing_network(df_to_draw):
        #adding the nodes to the network
        for j in range(numNodes):
            my_network.add_node(j, shape="dot")  
        
        #adding weight to the network
        for l in range(len(df_to_draw)):
            source_node = int(df_to_draw['source'][l])
            target_node = int(df_to_draw['target'][l])
            stake = int(df_to_draw['weight'][l])
            my_network.add_edge(source_node,target_node, title=stake, value=stake)
    
        #specifying the network's parameters
        my_network.repulsion(node_distance=500, central_gravity=0.1, spring_length=500, spring_strength=0.05, damping=0.1)
        my_network.heading = "Network of channels and stakes"
        
        return my_network.show('example.html')

In [77]:
drawing_network(df_initial)

In [78]:
drawing_network(df_recalculated)